In [10]:
import random
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from msedge.selenium_tools import Edge, EdgeOptions
from bs4 import BeautifulSoup
import getpass
import smtplib
from email.message import EmailMessage
from email.utils import formataddr
from dotenv import load_dotenv
import os
import datetime


# Constants
COUNTRY_URNS = {
    "Pakistan": "101165590",
    "United Arab Emirates": "104305776",
    "United States": "101022442",
    "India": "102713980",
    "United Kingdom": "103644278",
    "England": "103623778",
    "Portugal": "106808692",
    "Sri Lanka": "105146118",
    "Argentina": "104677530",
    "Finland": "100364837",
    "Saudi Arabia": "100456013",
    "Colombia": "104514075",
    "Canada": "102299470",
    "Afghanistan": "101240012",
    "Germany": "102454443",
    "Japan": "106215326",
    "Australia": "100446352",
    "Israel": "101452733",
    "Russia": "105117694",
    "Costa Rica": "102264497",
    "Iran": "101728296",
    "Turkey": "103121230",
    "Ukraine": "106714035",
    "Singapore": "103710677",
    "Netherlands": "103239229",
    "China": "104170880",
    "Philippines": "103619019",
    "Kuwait": "106155005",
    "Mexico": "100459316",
    "Italy": "100446943",
    "Oman": "106057199",
    "Jordan": "102890719",
    "Norway": "106693272",
    "Qatar": "103350119",
    "Greece": "105015875",
    "France": "101282230",
    "Sweden": "101174742",
    "Thailand": "103819153",
    "South Korea": "105149562",
    "Brazil": "101355337",
    "Egypt": "102890883",
    "Bangladesh": "100876405",
    "Switzerland": "101934083",
    "Syria": "102105699",
    "Malaysia": "101620260",
    "Denmark": "104514075"
}


# Introducing Random Sleep to mimic human behavior
def humanize_bot():
    time_to_sleep = random.randint(3, 7)
    time.sleep(time_to_sleep)

# Load environment variables
load_dotenv(".env")
sender_email = os.getenv("EMAIL")
password_email = os.getenv("PASSWORD")

# Initialize WebDriver
def initialize_driver(path):
    options = EdgeOptions()
    options.use_chromium = True
    return Edge(executable_path=path, options=options)

def login_to_linkedin(driver, username, password):
    driver.get('https://www.linkedin.com/login')
    driver.find_element(By.ID, 'username').send_keys(username)
    driver.find_element(By.ID, 'password').send_keys(password)
    driver.find_element(By.XPATH, "//button[@type='submit']").click()
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, 'global-nav-typeahead')))

def generate_linkedin_search_url(countries, page=1):
    geoUrns = "%2C".join([f"%22{COUNTRY_URNS[country]}%22" for country in countries])
    base_url = "https://www.linkedin.com/search/results/people/"
    return f"{base_url}?geoUrn=%5B{geoUrns}%5D&origin=FACETED_SEARCH&page={page}&sid=3UC"

def scroll_page(driver):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    WebDriverWait(driver, 30).until(lambda d: d.execute_script('return document.readyState') == 'complete')

def get_people_links(driver):
    WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.app-aware-link')))
    cards = driver.find_elements(By.CSS_SELECTOR, '.app-aware-link')
    return {card.get_attribute('href') for card in cards if '/in/' in card.get_attribute("href")}

def extract_details(driver):
    try:
        # Extracting email
        contact_info_button = WebDriverWait(driver, 40).until(EC.presence_of_element_located((By.LINK_TEXT, 'Contact info')))
        contact_info_button.click()
        email_element = WebDriverWait(driver, 40).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".pv-contact-info__ci-container.t-14 a[href^='mailto:']")))
        email = email_element.get_attribute("href").replace("mailto:", "")
    except (TimeoutException, NoSuchElementException):
        email = 'N/A'
    
    # Extracting location
    try:
        location_element = driver.find_element(By.CSS_SELECTOR, 'div.pv-text-details__left-panel span.text-body-small.inline.t-black--light.break-words')
        location = location_element.text.strip()
    except NoSuchElementException:
        location = 'N/A'
    
    # Extracting name
    try:
        page_source = BeautifulSoup(driver.page_source, "html.parser")
        name_element = page_source.find('h1', class_='text-heading-xlarge inline t-24 v-align-middle break-words')
        name = name_element.get_text(strip=True) if name_element else 'N/A'
    except Exception as e:
        name = 'N/A'
    
    return name, location, email

def send_email(subject, receiver_email, name):
    # Create the base text message.
    msg = EmailMessage()
    msg["Subject"] = subject
    msg["From"] = formataddr(("Empire Suites", f"{sender_email}"))
    msg["To"] = receiver_email
    msg["BCC"] = sender_email

    msg.set_content(
        f"""\
        {datetime.datetime.now().strftime('%d-%B-%Y (%I:%M %p)')}

        Dear {name},

        Empire Suites at Jumeirah Village Circle (JVC).
        Branded Luxury Apartments - Pay 10% to book!

        Escrow Account is open for transactions. 

        Introducing Empire Suites: Elevate your lifestyle to unprecedented heights in JVC. Immerse yourself in a world of sophistication, where premium finishes meet the allure of a luxury hotel apartment living experience.

        ✅ Luxury Studio, 1, and 2 bedroom apartments
        ✅ Ensuite private swimming pool and cinema included in every unit 
        ✅ Exquisitely furnished apartments
        ✅ Italian finishes and custom marble fittings 
        ✅ Exceptionally fitted kitchen with German appliances 
        ✅ Cutting-edge Smart home system
        ✅ Starting from AED 850K
        ✅ Flexible 60/40 Payment Plan
        ✅ Handover in December 2025

        Amenities:
        🏰 Magnificent 8000+ SQft lobby and reception with a business lounge and restaurant 
        🏊‍♀ Olympic-sized adults swimming pool
        🧒 Separate and secure kids pool
        💪 State-of-the-Art Gymnasium
        🎦 Full featured outdoor Cinema
        🧖‍♀ Luxurious Sauna
        🌋Invigorating Jacuzzi
        👶 Enchanting kids play area 
        🛥 20 mins to downtown and Dubai Canal 
        ✈ 18 mins to Dubai International Airport 
        🏬 Established community centre with a wide range of shops and restaurants 
        📍 Premium location with access to schools, medical facilities and malls 
        🌳 Lush green spaces to enjoy, with trails for jogging, cycling and walking 
        💼 Convenient Business Centre
        ...and much more! ✨

        🚧 *Construction Already Underway! * 🚧

        Don't miss out on this exclusive opportunity to experience unmatched opulence at Empire Suites! Limited inventory available, so ACT NOW!

        Get in touch today to secure your dream unit at Empire Suites and enjoy a lifetime of luxury and sophistication.
        """
    )
    
    
    try:
        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:  
            smtp.login(sender_email, password_email)  
            smtp.send_message(msg)  
            print(f"Email sent to {receiver_email} successfully!")  
    except Exception as e:
        print(f"Error sending email to {receiver_email}: {str(e)}")

if __name__ == "__main__":
    driver_path = "D:\\Data Science\\edgedriver_win64\\msedgedriver.exe"
    driver = initialize_driver(driver_path)
    humanize_bot()

    # Login to LinkedIn
    username = os.getenv("LINKEDIN_USERNAME")
    password = os.getenv("LINKEDIN_PASSWORD")

    login_to_linkedin(driver, username, password)
    humanize_bot()

    # Prompting user for country
    print("Available countries:", ", ".join(COUNTRY_URNS.keys()))
    selected_country = input("Enter the country you want to select from the list above: ")
    
    while selected_country not in COUNTRY_URNS:
        print("Invalid selection. Please select a valid country from the list.")
        selected_country = input("Enter the country you want to select from the list above: ")

    countries_to_filter = [selected_country]
    search_url = generate_linkedin_search_url(countries_to_filter)
    driver.get(search_url)
    humanize_bot()

    # Scroll page and get people links
    scroll_page(driver)
    people_links = get_people_links(driver)
    humanize_bot()

    # Loop through each link, extract details, and send email
    # Loop through each link, extract details, and send email
    for link in people_links:
        start_time = time.time()  # Start recording time
    
        driver.get(link)
        humanize_bot()

        name, location, email = extract_details(driver)
        print("-" * 90)
        print(f"Extracted Details:\nName: {name}\nLocation: {location}\nEmail: {email}\n")
    
        if email != 'N/A':
            try:
                print(f"Sending email to {email}...")
                send_email("Empire Suites Exclusive Offer", email, name)
                print(f"Email sent successfully to {email}!")
            except Exception as e:
                print(f"Error sending email to {email}: {str(e)}")
    
        humanize_bot()

        end_time = time.time()  # End recording time
        elapsed_time = end_time - start_time
        print(f"Time taken for {name} ({email}): {elapsed_time:.2f} seconds")


    driver.quit()

    
#hammadasif308@gmail.com
#pubg40241'

C:\Users\Noman Traders\AppData\Local\Temp\ipykernel_16924\1258302065.py:84: DeprecationWarning: Selenium Tools for Microsoft Edge is deprecated. Please upgrade to Selenium 4 which has built-in support for Microsoft Edge (Chromium): https://docs.microsoft.com/en-us/microsoft-edge/webdriver-chromium/#upgrading-from-selenium-3
  return Edge(executable_path=path, options=options)


Available countries: Pakistan, United Arab Emirates, United States, India, United Kingdom, England, Portugal, Sri Lanka, Argentina, Finland, Saudi Arabia, Colombia, Canada, Afghanistan, Germany, Japan, Australia, Israel, Russia, Costa Rica, Iran, Turkey, Ukraine, Singapore, Netherlands, China, Philippines, Kuwait, Mexico, Italy, Oman, Jordan, Norway, Qatar, Greece, France, Sweden, Thailand, South Korea, Brazil, Egypt, Bangladesh, Switzerland, Syria, Malaysia, Denmark
Enter the country you want to select from the list above: India
------------------------------------------------------------------------------------------
Extracted Details:
Name: Vishakha Chavan
Location: Greater Vadodara Area
Email: vishakha1306@gmail.com

Sending email to vishakha1306@gmail.com...
Email sent to vishakha1306@gmail.com successfully!
Email sent successfully to vishakha1306@gmail.com!
Time taken for Vishakha Chavan (vishakha1306@gmail.com): 18.71 seconds


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: MicrosoftEdge=118.0.2088.46)
